# 函数返回值规约
常见的规则：如果改变了入参，则返回None，反之，返回有意义的数字，如`list.sort`和`sorted`。除了`pop`函数

In [1]:
a = [1, 3, 2, 5, 7, 6]
print(sorted(a))
print(a)
print(a.sort())
print(a)
print(a.__delitem__(0))
print(a.pop())

[1, 2, 3, 5, 6, 7]
[1, 3, 2, 5, 7, 6]
None
[1, 2, 3, 5, 6, 7]
None
7


# 函数可读性
## 函数的注释及doctest及splinx（见课件）
为`__doc__`属性，`help`方法包含了函数定义和函数的`__doc__`

In [5]:
def factorial(n: 'int >= 0') -> int:
    """返回n!"""
    return a if n < 2 else n * factorial(n-1)
print(factorial.__doc__)
print(factorial.__annotations__)
help(factorial)

返回n!
{'n': 'int >= 0', 'return': <class 'int'>}
Help on function factorial in module __main__:

factorial(n: 'int >= 0') -> int
    返回n!



## 函数的参数类型定义
1. 通过注释定义
2. 通过python3的形式定义

In [6]:
def factorial(n: int) -> int:
    """返回n!"""
    return a if n < 2 else n * factorial(n-1)

### 类型提示（见ppt）

# 高阶函数
接受函数作为参数

In [7]:
card_order = [str(x) for x in range(3, 11)] + list("JQKA2")
cards = ["2", "3", "7", "J", "A"]
print(sorted(cards))
print(sorted(cards, key=card_order.index))

['2', '3', '7', 'A', 'J']
['3', '7', 'J', 'A', '2']


## 高阶函数替换法
* map filter，可以用列表推导/生成器表达式替换
* lambda都可以用函数替换

In [8]:
sorted(cards, key=lambda x: card_order.index(x))

['3', '7', 'J', 'A', '2']

## 使用高阶函数实现factorial

In [10]:
from functools import reduce
def factorial(n):
    return reduce(lambda a, b: a * b, range(1, n+1))
print(factorial(5))
from operator import mul
def factorial(n):
    return reduce(mul, range(1, n+1))

120


## 装饰器与闭包
* 什么是装饰器？参数是被装饰函数的函数
* 什么功能？给被装饰提供额外的功能，或产生全新的函数
* 什么时候起作用？在文件被import时
* 什么时候用？
    * 场景1：需要记录额外信息，如动态注册

In [16]:
def nochange_decorate(func):
    return func

@nochange_decorate
def target():
    print("running target")
# 等价于 target = nochange_decorate(target)
target()

def complete_change_decorate(func):
    def inner():
        print("running inner")
    return inner

@complete_change_decorate
def target():
    print("running target")
target()

running target
running inner


### 变量的作用域——`global` vs `nonlocal` vs 局部变量 vs 自由变量

In [29]:
print(__builtins__.__dir__)
def f1(a):
    print(a)
    print(b)
f1(3)

<built-in function __dir__>
3
9


In [20]:
b = 6
f1(3)

3
6


In [22]:
def f2(a):
    print(a)
    print(b)
    b = 9
f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [34]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)

3
9


In [36]:
# 计算平均值
def make_average():
    """

    >>>make_average()
    >>>avg = make_average()
    >>>avg(10)
    10.0
    >>>avg(11)
    10.5
    """
    pass


In [43]:
def make_average():
    count, total = 0, 0
    def average(val):
        count += 1
        total += val
        return total / count
    return average
avg = make_average()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

* 函数内赋值的变量局部变量，除非：
    * 使用`global`标记为全局变量
    * 使用`nonolocal`标记为自由变量
* 依赖全局变量是危险的

In [44]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('val', 'count', 'total')
()


In [59]:
def make_average():
    count, total = 0, 0
    def average(val):
        nonlocal count, total
        count += 1
        total += val
        return total / count
    return average
avg = make_average()
print(avg(10))
print(avg.__closure__)
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

10.0
(<cell at 0x1089d7e50: int object at 0x1063fad78>, <cell at 0x1089b96f0: int object at 0x1063fae98>)
('val',)
('count', 'total')


#### functools.wraps(func)保留函数的__name__和__doc__

In [60]:
import time
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*arg, **kwargs):
        t0 = time.time()
        result = func(*arg, **kwargs)
        print(f'{func.__name__} cost {time.time() - t0}')
        return result
    return clocked

@clock
def factorial(n):
    return reduce(lambda a, b: a * b, range(1, n+1))


print(factorial(1000))
help(factorial)

factorial cost 0.00042700767517089844
40238726007709377354370243392300398571937486421071463254379991042993851239862902059204420848696940480047998861019719605863166687299480855890132382966994459099742450408707375991882362772718873251977950595099527612087497546249704360141827809464649629105639388743788648733711918104582578364784997701247663288983595573543251318532395846307555740911426241747434934755342864657661166779739666882029120737914385371958824980812686783837455973174613608537953452422158659320192809087829730843139284440328123155861103697680135730421616874760967587134831202547858932076716913244842623613141250878020800026168315102734182797770478463586817016436502415369139828126481021309276124489635992870511496497541990934222156683257208082133318611681155361583654698404670897560290095053761647584772842188967964624494516076535340819890138544248798495995331910172335555660213945039973628075013783761530712776192684903435262520001588853514733161170210396817592151090778801939317811419454525

### classmethod, staticmethod, property
* `classmethod`，提供类的其他初始化方法
* `staticmethod`，静态方法，与类状态无关
* `property`，惰性属性

In [67]:
import dataclasses

@dataclasses.dataclass
class Cards:
    suits: str
    rank: str
    @property
    def score(self):
        return 1
    def __setattr__(self, __name: str, __value) -> None:
        pass
card = Cards("♣️", "A")

card.score
card.score = 3
print(card.score)

1
